In [2]:
import sqlite3

# Conexión a la base de datos (creará el archivo si no existe)
conn = sqlite3.connect("app.db")
cursor = conn.cursor()

# Activar claves foráneas en SQLite
cursor.execute("PRAGMA foreign_keys = ON;")

# Tabla usuarios
cursor.execute("""
CREATE TABLE IF NOT EXISTS usuarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL,
    fecha_registro DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

# Tabla cursos
cursor.execute("""
CREATE TABLE IF NOT EXISTS cursos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    codigo TEXT UNIQUE NOT NULL,               -- Ej: "C1", "C2"
    titulo TEXT NOT NULL,
    descripcion TEXT,
    nivel TEXT,                                -- Ej: 'principiante', 'intermedio', 'avanzado'
    duracion INTEGER,                          -- En minutos
    precio REAL DEFAULT 0.0,
    fecha_creacion DATETIME DEFAULT CURRENT_TIMESTAMP,
    rating_promedio REAL DEFAULT 0.0,
    activo BOOLEAN DEFAULT 1
);
""")

# Tabla curso_temas
cursor.execute("""
CREATE TABLE IF NOT EXISTS curso_temas (
    curso_id INTEGER NOT NULL,
    tema TEXT NOT NULL,
    FOREIGN KEY (curso_id) REFERENCES cursos(id) ON DELETE CASCADE
);
""")

# Tabla carrito
cursor.execute("""
CREATE TABLE IF NOT EXISTS carrito (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    usuario_id INTEGER NOT NULL,
    curso_id INTEGER NOT NULL,
    fecha_agregado DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (usuario_id) REFERENCES usuarios(id),
    FOREIGN KEY (curso_id) REFERENCES cursos(id)
);
""")

# Tabla compras
cursor.execute("""
CREATE TABLE IF NOT EXISTS compras (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    usuario_id INTEGER NOT NULL,
    total REAL NOT NULL,
    fecha_compra DATETIME DEFAULT CURRENT_TIMESTAMP,
    voucher TEXT,
    FOREIGN KEY (usuario_id) REFERENCES usuarios(id)
);
""")

# Tabla detalle_compra
cursor.execute("""
CREATE TABLE IF NOT EXISTS detalle_compra (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    compra_id INTEGER NOT NULL,
    curso_id INTEGER NOT NULL,
    precio_pagado REAL NOT NULL,
    FOREIGN KEY (compra_id) REFERENCES compras(id),
    FOREIGN KEY (curso_id) REFERENCES cursos(id)
);
""")

# Tabla ofertas
cursor.execute("""
CREATE TABLE IF NOT EXISTS ofertas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    curso_id INTEGER NOT NULL,
    descuento_porcentaje REAL NOT NULL,
    fecha_inicio DATETIME NOT NULL,
    fecha_fin DATETIME NOT NULL,
    aprobada BOOLEAN DEFAULT 0,
    FOREIGN KEY (curso_id) REFERENCES cursos(id)
);
""")

# Guardar cambios y cerrar conexión
conn.commit()
conn.close()

print("Base de datos creada correctamente.")


Base de datos creada correctamente.


In [3]:
import sqlite3
import json
import random

# Conectar a la base de datos
conn = sqlite3.connect("app.db")
cursor = conn.cursor()

# Cargar el JSON
with open("cursos.json", "r", encoding="utf-8") as f:
    cursos_data = json.load(f)

# Insertar los datos
for curso in cursos_data:
    codigo = curso["id"]
    titulo = curso["titulo"]
    descripcion = curso["descripcion"]
    nivel = curso["nivel"]
    duracion = random.choice([30, 45, 60, 75, 90, 105, 120, 135, 150])
    precio = random.choice([20, 30, 40, 50])
    rating_promedio = round(random.uniform(3.5, 4.5), 1)
    activo = 1

    # Insertar en tabla cursos
    cursor.execute("""
        INSERT INTO cursos (codigo, titulo, descripcion, nivel, duracion, precio, rating_promedio, activo)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (codigo, titulo, descripcion, nivel, duracion, precio, rating_promedio, activo))

    curso_id = cursor.lastrowid  # ID autogenerado del curso recién insertado

    # Insertar los temas relacionados
    for tema in curso["temas"]:
        cursor.execute("""
            INSERT INTO curso_temas (curso_id, tema)
            VALUES (?, ?)
        """, (curso_id, tema))

# Guardar los cambios y cerrar conexión
conn.commit()
conn.close()

print("✅ Cursos y temas insertados correctamente.")


✅ Cursos y temas insertados correctamente.
